In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def logic_win(x,x_r):
    return (x >= x_r[0]) & (x < x_r[1])

import warnings
warnings.filterwarnings('ignore')

#### Import and clean dataset, select out London

In [2]:
path = 'Dataframes/combined_collisions_v3.csv'

df = pd.read_csv(path,low_memory=False)
df.drop(columns='Unnamed: 0',inplace=True)

In [3]:
#Time column has some nans in it, must drop those entries

def is_str(x):
    Nx = len(x)
    I = np.ones(Nx).astype(bool)
    for ii in range(Nx):
        if type(x[ii]) != str:
            I[ii] = False
    return I

df = df.loc[is_str(df.Time.values),:]
df.aadf_Count_point_id = df.aadf_Count_point_id.values.astype(int)

In [4]:
#make hour, year and month columns
    #hour will be rounded 
def get_hour(T):
    Nt = len(T)
    H = np.zeros(Nt).astype(int)
    for ii in range(Nt):
        s = T[ii].split(':')
        h = int(s[0])
        m = int(s[1])
        if m > 30:
            h += 1
        H[ii] = h % 24
    return H

def get_ymd(D):
    Nd = len(D)
    # |
    ymd = np.zeros((Nd,3)).astype(int)
    for ii in range(Nd):
        s = D[ii].split('-')
        for nn in range(3):
            ymd[ii,nn] = int(s[nn])
    return ymd

def reformat_aadf_year(Year):
    Ny = len(Year)
    y = np.zeros(Ny).astype(int)
    for ii in range(Ny):
        y[ii] = int(Year[ii].split('-')[0])
    return y

df[['y','m','d']] = get_ymd(df.Date.values)
df['h'] = get_hour(df.Time.values)
#df['aadf_Year'] = reformat_aadf_year(df.aadf_Year.values)

##### Below selects out region. We select greater London. Drop accident entries with no matches to a traffic counter
- user can select our different regions by logically filtering with latitude and logitude

In [5]:
#select Greater London as indicated by feature in_london
df_lond = df.loc[df.in_london,:].copy() #this selects Greater London

#drop all data that don't have traffic matches
I = df_lond.match == True
df_lond = df_lond.loc[I,:]
df_lond.aadf_Year = reformat_aadf_year(df_lond.aadf_Year.values)

#### Add and clean features

In [6]:
#Make a feature for if there is a bike lane of not ('bikelane'). There is no bike lane if all
#the below classifiers are False

f_laneClass = ['aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME']

Inolane = np.ones(df_lond.shape[0]).astype(bool)
for f in f_laneClass:
    Inolane &= (df_lond[f].values == False)
    
df_lond['bikelane'] = ~Inolane

In [7]:
#clean up aadf_BOROUGH feature. Select mode from the list. Assign "NA" if no boroughs or none is listed

def clean_borough(borough_vals):
    Ndf = borough_vals.shape[0]
    bor_cleaned = np.zeros(Ndf).astype(str)
    for nn,Astr in enumerate(borough_vals):
        Spl1 = Astr.split('[')[2:]
        Spl2 = Spl1[0].split(',')

        Nspl = len(Spl2)
        Spl2[-1] = Spl2[-1].split(']]')[0]
        for ii,s in enumerate(Spl2):
            split = s.split("'")
            if len(split) > 1:
                Spl2[ii] = s.split("'")[1]
            else:
                Spl2[ii] = 'NA'
        bor_cleaned[nn] = pd.DataFrame(Spl2).mode().values[0][0]
    return bor_cleaned

df_lond['aadf_BOROUGH'] = clean_borough(df_lond.aadf_BOROUGH.values)

#### Create df_road
- we now take the combined traffic-accident dataframe and form a new dataframe where the entries are now roads (traffic counters)
- we add features indicating bike acccident totals for the road
- lastly we add in roads that had no recorded accidents
- some aadf features are averaged over for accidents with shared road (such as traffic volumes)
- for most features we take the mode. The vast majority of these features will be single valued across all accidents on a given road

In [9]:
traf_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().index)
Ninc_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().values)

######################################
Nthresh = 1 #this sets which traffic counters are included, they must have at least this many accidents 
    #typically we want to leave this as 1, especially since we will include roads with 0 accidents below
######################################

I = Ninc_IDs >= Nthresh #this will select roads to include in the new DataFrame based on if they had at least this many accidents
I.sum()

IDs_df = traf_IDs[I]
Nid = len(IDs_df)

df_road = pd.DataFrame({'ID':IDs_df})
slight_rd = np.zeros(Nid).astype(int)
ser_rd = np.zeros(Nid).astype(int)
total_rd = np.zeros(Nid).astype(int)
for nn,ii in enumerate(IDs_df):
    I = df_lond.aadf_Count_point_id.values == ii
    total_rd[nn] = I.sum()
    slight_rd[nn] = df_lond.loc[I,'slight'].values.sum()
    ser_rd[nn] = total_rd[nn] - slight_rd[nn]
df_road['total'] = total_rd
df_road['slight'] = slight_rd
df_road['serious'] = ser_rd

safety_mode = ['Road_name','Road_name2','h','Day_of_Week','m','Speed_limit','Junction_Detail','Junction_Control','Pedestrian_Crossing-Physical_Facilities']
aadf_av = ['distance_to_cp','aadf_Pedal_cycles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
aadf_mode = ['bikelane','aadf_Road_name','aadf_Road_category','aadf_Road_type','aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME','aadf_Link_length_miles','aadf_BOROUGH','aadf_Latitude','aadf_Longitude']

for f in safety_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]

for f in aadf_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]
        
for f in aadf_av:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mean()
        
median_d = df_road.distance_to_cp.median()
median_bv = df_road.aadf_Pedal_cycles.median()
df_road['total_dnorm'] = df_road['total'].values*median_d/df_road['distance_to_cp'].values
df_road['total_dnorm_bvnorm'] = df_road.total_dnorm.values*median_bv/df_road.aadf_Pedal_cycles.values
df_road['serious_ratio'] = (df_road['serious'].values)/df_road['total'].values

##### Merge in roads that had no accidents (this will only work for Greater London roads)

In [11]:
df_allroad = pd.read_csv('Dataframes/aadf_features_london_colyears.csv')
#here we read in a aadf dataframe of Greater London roads with data that span the years in the bike accident dataset

In [12]:
all_IDs = df_allroad.Count_point_id.value_counts().index

#find the IDs that are missing in above df_road, these are the ones with no accidents
Izero = ~np.isin(all_IDs,traf_IDs)
zero_IDs = all_IDs[Izero]

In [13]:
#now go through and make a df formatted exactly the same as df_road for the zero accident roads

road_cols = np.array(df_road.columns)
#get features that are already in df_allroads
allroad_cols = df_allroad.columns
allroad_colsrename = {}
for ii,f in enumerate(allroad_cols):
    allroad_colsrename[f] = 'aadf_'+f
    
df_allroad.rename(allroad_colsrename,inplace=True,axis='columns')
df_allroad.rename({'aadf_Count_point_id':'ID'},inplace=True,axis='columns')

In [14]:
#need to clean aadf_BOROUGH here as well
df_allroad.aadf_BOROUGH = clean_borough(df_allroad.aadf_BOROUGH.values)

In [15]:
df_road_zero = pd.DataFrame()
df_road_zero['ID'] = zero_IDs
Nzero = Izero.sum()
Inotin = ~np.isin(road_cols,df_allroad.columns)
fzeros = np.array(['total','slight','serious','total_dnorm','total_dnorm_bvnorm'])
for f in road_cols[Inotin]:
    if np.isin(f,fzeros):
        df_road_zero[f] = np.zeros(Nzero)
    else:
        df_road_zero[f] = -9*np.ones(Nzero)
    

f_mode = ['aadf_Road_name','aadf_Road_category','aadf_Road_type','aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME','aadf_Latitude','aadf_Longitude','aadf_BOROUGH'] 
f_av = ['aadf_Link_length_miles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
for f in f_mode:
    A = [None]*Nzero
    for ii,rid in enumerate(zero_IDs):
        I = df_allroad.ID == rid
        A[ii] = df_allroad.loc[I,f].mode().values[0]
    df_road_zero[f] = A
    
for f in f_av:
    A = [None]*Nzero
    for ii,rid in enumerate(zero_IDs):
        I = df_allroad.ID == rid
        A[ii] = df_allroad.loc[I,f].mean()
    df_road_zero[f] = A
    
Inan = np.isnan(df_road_zero.aadf_Link_length_miles.values)
df_road_zero.loc[Inan,'aadf_Link_length_miles'] = -9

In [16]:
f_inttype = ['total', 'slight', 'serious', 'Road_name', 'Road_name2',
       'h', 'Day_of_Week', 'm', 'Speed_limit', 'Junction_Detail',
       'Junction_Control', 'Pedestrian_Crossing-Physical_Facilities']
f_str_accident = ['Road_name','Road_name2']

for f in f_inttype:
    df_road_zero[f] = df_road_zero[f].values.astype(int)
    
for f in f_str_accident:
    df_road_zero[f] = 'N/A'

In [17]:
#lastly need to set 'bikelane feature'

f_laneClass = ['aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME']

Inolane = np.ones(df_road_zero.shape[0]).astype(bool)
for f in f_laneClass:
    Inolane &= (df_road_zero[f].values == False)
    
df_road_zero['bikelane'] = ~Inolane

##### Now we can concatenate df_road_zero into df_road

In [18]:
df_road = pd.concat([df_road,df_road_zero],ignore_index=True)

##### Add metadata for survey years

In [19]:
#add in aadf year availability metadata 
path_year = 'Dataframes/aadf_year_meta.csv' #this has extracted metadata for all roads that give survey year availability
df_year = pd.read_csv(path_year)

Nroadall = df_road.shape[0]

#merge the metadata into df_road
feat_add = list(df_year.columns)
feat_add.remove('Count_point_id')
for c in feat_add:
    df_road[c] = np.zeros(Nroadall).astype(int)
    for ii,ID in enumerate(df_road.ID.values):
        I = df_year.Count_point_id == ID
        df_road.loc[ii,c] = df_year.loc[I,c].values

#### df_road is complete. Save the dataframe

In [20]:
df_road.sample(5)

ID  total  slight  serious Road_name Road_name2   h  Day_of_Week  \
31      6802    105      91       14      A235          U   8            5   
1210   17620      7       7        0     A4140          C  14            1   
3802  942468      0       0        0       N/A        N/A  -9           -9   
1375   78366      5       4        1     A1069          U  10            1   
176    56965     51      41       10     A3036          U   8            3   

       m  Speed_limit  ...  aadf_All_HGVs    aadf_LGVs  \
31    10           30  ...     501.733333  2376.380952   
1210   5           30  ...     152.714286   993.428571   
3802  -9           -9  ...      79.000000    58.000000   
1375   4           30  ...     190.400000  1117.000000   
176    8           30  ...     903.117647  4020.411765   

      aadf_Buses_and_coaches total_dnorm total_dnorm_bvnorm serious_ratio  \
31                818.895238   23.046465          24.637612      0.133333   
1210              217.714286    1.662894          14.274630      0.000000   
3802                0.000000    0.000000           0.000000     -9.000000   
1375              937.600000    3.857361          13.827155      0.200000   
176              1457.686275   40.773477           3.001594      0.196078   

     N_year Year_start Year_end Max_year_gap  
31       22       2000     2021            1  
1210     22       2000     2021            1  
3802      1       2008     2008            1  
1375     22       2000     2021            1  
176      22       2000     2021            1  

[5 rows x 49 columns]

In [21]:
#save dataframe

path = 'Dataframes/df_road_v3p1_zeros.csv'
df_road.to_csv(path,index_label=False)